## Proyecto Marketing Analytics para empresa Fintech

## DESCRIPCIÓN DEL PROYECTO

Análisis de los datos de campañas de marketing de una empresa Fintech para
conseguir identificar patrones, tendencias y factores que influyen en que un
cliente acabe contratando un depósito. Tras un análisis exhaustivo de todos sus
datos, habrá que implementar un modelo simple que ayude a entender y predecir
la efectividad de las campañas de marketing, y diseñar un dashboard para la
capa ejecutiva de la fintech con los principales insights y KPIs de los resultados
de las campañas.

In [ ]:
# Manipulación y análisis de datos
import pandas as pd
import numpy as np

# Visualización de datos
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import utils

# Expresiones regulares
import re
from fuzzywuzzy import process

In [ ]:
df = pd.read_csv('bank-additional_bank-additional-full.csv', sep=';')
df

## 1.Análisi de datos

### Revisión general de los datos

In [ ]:
utils.check_df(df)

### DATOS --> Valores nulos y/o faltantes y duplicados

In [ ]:
utils.identificacion_valores_problem(df)

## Pregunta de negocio

¿Qué factores influyen en que un cliente contrate un depósito (y = yes) y cómo podemos optimizar las campañas para aumentar las contrataciones?

Variable objetivo

y → contratación del depósito (yes / no).

## Vista general

In [ ]:
# tasa de conversión

df.y.value_counts()

In [ ]:
sns.countplot(x='y', data=df)
plt.title('Distribución de la variable objetivo')
plt.xlabel('Contratación de depósito')
plt.ylabel('Número de clientes')
plt.show()

In [ ]:
Conversión= df.y.value_counts(normalize=True) * 100
Conversión


In [ ]:

df_conv = Conversión.reset_index()
df_conv.columns = ['y', 'porcentaje']

sns.barplot(x='y', y='porcentaje', data=df_conv)
plt.title('Tasa de conversión de depósitos (%)')
plt.ylabel('Porcentaje (%)')
plt.xlabel('Contratación')
plt.ylim(0, 100)
plt.show()

Observamos que solo el 11,27% se los contactos acaban con la contratación de ls depósitos

## General

In [ ]:
print('Número de llamadas:', df.shape[0])

In [ ]:
df.campaign.mean()

In [ ]:
contacto_nominal = df.contact.value_counts()
contacto_pct = df.contact.value_counts(normalize=True)
contacto_comb= pd.merge(contacto_nominal,contacto_pct, on=['contact'])
contacto_comb


In [ ]:
# tasa de conversion segun la cantidad de llamadas
conv_contacto = df.groupby('contact')['y'].apply(lambda x: (x=='yes').mean()*100).reset_index()
conv_contacto.columns = ['contact','pct_contratación']
conv_contacto.sort_values(by='pct_contratación', ascending=False)

## Perfil clientes

In [ ]:
df_copy = df.copy()

## AGE

In [ ]:
px.histogram(df_copy, x="age", nbins=25)

In [ ]:

bins = [0, 25, 35, 45, 55, 65, 100]
labels = ['0-25', '25-35', '35-45', '45-55', '55-65', '+65']

df_copy['intervalo_edad'] = pd.cut(
    df_copy['age'],
    bins=bins, 
    labels = labels,
    right = False,
    ordered=True
)


In [ ]:
px.histogram(df_copy, 
             x='intervalo_edad',
             category_orders= {'intervalo_edad' : labels})


In [ ]:
conv_age = df_copy.groupby('intervalo_edad')['y'].apply(lambda x: (x=='yes').mean()*100).reset_index()
conv_age.columns = ['intervalo_edad','pct_contratación']
conv_age

In [ ]:
sns.barplot(x='intervalo_edad', y='pct_contratación', data=conv_age)
plt.title('Tasa de conversión por segmento')
plt.ylabel('Porcentaje (%)')
plt.xlabel('Segmento')
plt.show()

In [ ]:
df[df.age >= 65] # eliminar outliers?

Al analizar la tasa de conversión por grupos de edad, se observa que los clientes más jóvenes (18-25) y los mayores de 65 años son los que presentan **mayor porcentaje de contratación de depósitos**. 

Sin embargo, estos mismos segmentos son los que **menos contactos reciben** durante las campañas, lo que indica una oportunidad clara de mejora. Incrementar de manera controlada la frecuencia de contacto con estos grupos podría traducirse en un **aumento significativo de las contrataciones** sin necesidad de aumentar el volumen total de clientes contactados.

## Job

In [ ]:
sns.countplot(y='job', data=df_copy)

In [ ]:
job_nominal = df_copy.groupby('job')['y'].value_counts()
job_nominal

In [ ]:
job_pct= df_copy.groupby('job')['y'].value_counts(normalize=True)*100
job_pct

In [ ]:
combinado = pd.merge(job_nominal,job_pct, on=['job','y'])
combinado

In [ ]:
#solo coger los si
conv_job = df_copy.groupby('job')['y'].apply(lambda x: (x=='yes').mean()*100).reset_index()
conv_job.columns = ['job','pct_contratación']
conv_job.sort_values(by='pct_contratación', ascending=False)

In [ ]:
sns.barplot(y='job', x='pct_contratación', data=conv_job)
plt.title('Tasa de conversión por segmento')
plt.ylabel('Porcentaje (%)')
plt.xlabel('Segmento')
plt.show()

Al analizar la tasa de conversión por tipo de trabajo, se observa que ciertos segmentos presentan una mayor propensión a contratar depósitos. 

En particular, los **estudiantes** y los **jubilados** muestran los **porcentajes más altos de contratación**, mientras que otros trabajos como los **empleados de servicios** o los **blue collar** presentan tasas de conversión más bajas. 

Esto indica que los esfuerzos de contacto deberían enfocarse en los segmentos con mayor propensión, especialmente aquellos que históricamente reciben menos contactos, para **maximizar la efectividad de las campañas de marketing**.

In [ ]:
sns.boxplot(x='age', y='job', data=df_copy)

## Numero de contactos 'Campaign'

la columna campaña nos indica cuantas veces se ha contactado con el cliente durante la campaña.
Queremos saber cuanta cantidad de llamadas se han realizado a cada segmento de edad y de trabajo.
Con el objetivo de ver si hay déficit o si se puede aprovechar mas en algun sector.

In [ ]:
px.histogram(df_copy,x='campaign',nbins=100)

In [ ]:
#Contactos por grupo de edad

df_copy.groupby('intervalo_edad')['campaign'].mean()

In [ ]:
# contactos medio por trabajo
df_copy.groupby('job')['campaign'].mean()

Aun que los grupos de edades de entre 0-25 y  +65 son los en proporción mas contratan depositos, de media reciben el mismo numero de llamadas que todos los demas grupos de edades.
Por lo que no se necesita un numero de llamadas superior para que este grupo de edades contraten su servicio.

## Duracion de las llamadas

In [ ]:
# voy a quitar los outliers de la duration ya que pueden crear malentendidos
limite_inferior, limite_superior = utils.limites_outliers(df_copy,'duration')

In [ ]:
df_sin_outliers = df_copy[(df_copy['duration'] >= limite_inferior) & (df_copy['duration'] <= limite_superior)]

In [ ]:
sns.boxplot(x='duration', y='y', data=df_sin_outliers)

In [ ]:
pd.crosstab(df_sin_outliers['job'] , df_sin_outliers['duration'])

In [ ]:
pd.crosstab(df_sin_outliers['intervalo_edad'] , df_sin_outliers['duration'])

In [ ]:
plt.figure(figsize=(10,6))
sns.violinplot(x='intervalo_edad', y='duration', hue='y', data=df_sin_outliers, split=True, palette='Set2')
plt.title('Duración de la llamada por grupo de edad y resultado')
plt.ylabel('Duración de la llamada (segundos)')
plt.xlabel('Grupo de edad')
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
sns.violinplot(x='job', y='duration', hue='y', data=df_sin_outliers, split=True, palette='coolwarm')
plt.xticks(rotation=45)
plt.title('Duración de la llamada por trabajo y resultado')
plt.ylabel('Duración de la llamada (segundos)')
plt.xlabel('Trabajo')
plt.show()

## Cruces finales

In [ ]:
df_age_job = df_copy.groupby(['intervalo_edad','job'])['y'].apply(lambda x: (x=='yes').mean()*100).reset_index()
df_age_job_pivot = df_age_job.pivot(index='job', columns='intervalo_edad', values='y')

plt.figure(figsize=(12,6))
sns.heatmap(df_age_job_pivot, annot=True, fmt=".1f", cmap='YlGnBu')
plt.title('Tasa de conversión (%) por grupo de edad y tipo de trabajo')
plt.ylabel('Trabajo')
plt.xlabel('Grupo de edad')
plt.show()

In [ ]:
bins = [0,2,4,6]
labels = ['0-2','2-4','+5']

df_copy['intervalo_euribor'] = pd.cut(
    df_copy['euribor3m'],
    bins=bins, 
    labels = labels,
    right = False,
    ordered=True)

px.histogram(df_copy, 
             x='intervalo_euribor',
             category_orders= {'intervalo_euribor' : labels})

In [ ]:
estaciones = {
    'dec': 'Invierno', 'jan': 'Invierno', 'feb': 'Invierno',
    'mar': 'Primavera', 'apr': 'Primavera', 'may': 'Primavera',
    'jun': 'Verano', 'jul': 'Verano', 'aug': 'Verano',
    'sep': 'Otoño', 'oct': 'Otoño', 'nov': 'Otoño'
}
df_copy['estaciones'] = df_copy['month'].map(estaciones)

In [ ]:
pd.crosstab(df_copy['y'] , df_copy['job'], normalize=True)*100

In [ ]:
pd.crosstab(df_copy['y'] , df_copy['education'], normalize=True)*100

In [ ]:
pd.crosstab(df_copy['y'] , df_copy['marital'])

In [ ]:
pd.crosstab(df_copy['y'] , df_copy['contact'])

In [ ]:
plt.figure(figsize=(14, 6))
sns.boxplot(
    data=df,
    x='job',
    y='duration',
    hue='y'
)